<a href="https://colab.research.google.com/github/CassDabii/BBC-DS-Task/blob/main/BBC_DSProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd

bike_journey = pd.read_csv('BBCDatasets/bike_journeys.csv')

In [12]:
bike_journey.head()

,Journey Duration,Journey ID,End Date,End Month,End Year,End Hour,End Minute,End Station ID,Start Date,Start Month,Start Year,Start Hour,Start Minute,Start Station ID
0,2040.0,953,19,9,17,18.0,0.0,478.0,19.0,9.0,17.0,17.0,26.0,251.0
1,1800.0,12581,19,9,17,15.0,21.0,122.0,19.0,9.0,17.0,14.0,51.0,550.0
2,1140.0,1159,15,9,17,17.0,1.0,639.0,15.0,9.0,17.0,16.0,42.0,212.0
3,420.0,2375,14,9,17,12.0,16.0,755.0,14.0,9.0,17.0,12.0,9.0,163.0
4,1200.0,14659,13,9,17,19.0,33.0,605.0,13.0,9.0,17.0,19.0,13.0,36.0
